In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession

In [2]:
df=pd.read_csv("Pistachio_16_Features_Dataset.csv")
df=df.dropna()
df['AREA']=df['AREA'].astype(int)
from sklearn.preprocessing import LabelEncoder
lc=LabelEncoder()
df['Class']=lc.fit_transform(df['Class'])
df

,AREA,PERIMETER,MAJOR_AXIS,MINOR_AXIS,ECCENTRICITY,EQDIASQ,SOLIDITY,CONVEX_AREA,EXTENT,ASPECT_RATIO,ROUNDNESS,COMPACTNESS,SHAPEFACTOR_1,SHAPEFACTOR_2,SHAPEFACTOR_3,SHAPEFACTOR_4,Class
0,63391,1568.4050,390.3396,236.7461,0.7951,284.0984,0.8665,73160.0,0.6394,1.6488,0.3238,0.7278,0.0062,0.0037,0.5297,0.8734,0
1,68358,1942.1870,410.8594,234.7525,0.8207,295.0188,0.8765,77991.0,0.6772,1.7502,0.2277,0.7181,0.0060,0.0034,0.5156,0.9024,0
2,73589,1246.5380,452.3630,220.5547,0.8731,306.0987,0.9172,80234.0,0.7127,2.0510,0.5951,0.6767,0.0061,0.0030,0.4579,0.9391,0
3,71106,1445.2610,429.5291,216.0765,0.8643,300.8903,0.9589,74153.0,0.7028,1.9879,0.4278,0.7005,0.0060,0.0030,0.4907,0.9755,0
4,80087,1251.5240,469.3783,220.9344,0.8823,319.3273,0.9657,82929.0,0.7459,2.1245,0.6425,0.6803,0.0059,0.0028,0.4628,0.9833,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2143,85983,1157.1160,444.3447,248.8627,0.8284,330.8730,0.9823,87536.0,0.6799,1.7855,0.8070,0.7446,0.0052,0.0029,0.5545,0.9900,1
2144,85691,2327.3459,439.8794,278.9297,0.7732,330.3107,0.8886,96439.0,0.6590,1.5770,0.1988,0.7509,0.0051,0.0033,0.5639,0.8892,1
2145,101136,1255.6190,475.2161,271.3299,0.8210,358.8459,0.9888,102286.0,0.7584,1.7514,0.8061,0.7551,0.0047,0.0027,0.5702,0.9987,1
2146,97409,1195.2150,452.1823,274.5764,0.7945,352.1718,0.9902,98376.0,0.7635,1.6468,0.8569,0.7788,0.0046,0.0028,0.6066,0.9989,1


In [3]:
spark = SparkSession.builder.appName("PrincipalComponent").getOrCreate()

In [4]:
data1 = spark.createDataFrame(df)
data1

DataFrame[AREA: bigint, PERIMETER: double, MAJOR_AXIS: double, MINOR_AXIS: double, ECCENTRICITY: double, EQDIASQ: double, SOLIDITY: double, CONVEX_AREA: double, EXTENT: double, ASPECT_RATIO: double, ROUNDNESS: double, COMPACTNESS: double, SHAPEFACTOR_1: double, SHAPEFACTOR_2: double, SHAPEFACTOR_3: double, SHAPEFACTOR_4: double, Class: bigint]

In [5]:
cl=list(df.columns[0:16])

In [6]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=cl,outputCol="features")

In [7]:
d=assembler.transform(data1)
d = d.select(['features','Class'])
d.show()

+--------------------+-----+
|            features|Class|
+--------------------+-----+
|[63391.0,1568.405...|    0|
|[68358.0,1942.187...|    0|
|[73589.0,1246.538...|    0|
|[71106.0,1445.261...|    0|
|[80087.0,1251.524...|    0|
|[52268.0,1154.15,...|    0|
|[71693.0,1463.932...|    0|
|[62240.0,1898.45,...|    0|
|[64319.0,2011.291...|    0|
|[78115.0,1238.623...|    0|
|[69391.0,1930.618...|    0|
|[67309.0,2353.405...|    0|
|[81656.0,1197.543...|    0|
|[61725.0,1387.943...|    0|
|[81583.0,1295.336...|    0|
|[62862.0,1086.575...|    0|
|[84537.0,1181.153...|    0|
|[89306.0,1940.193...|    0|
|[70220.0,1217.227...|    0|
|[74843.0,1158.202...|    0|
+--------------------+-----+
only showing top 20 rows



In [8]:
train,test=d.randomSplit([0.8,0.2],seed=42)

In [9]:
from pyspark.ml.feature import StandardScaler
  
scaler = StandardScaler(inputCol="features", 
                        outputCol="scaledFeatures", 
                        withStd=True, 
                        withMean=False)
scalerModel = scaler.fit(d)
d= scalerModel.transform(d)
  
d.select('scaledFeatures').show(5)

+--------------------+
|      scaledFeatures|
+--------------------+
|[4.83099121255962...|
|[5.20952339146173...|
|[5.60817485669969...|
|[5.41894687195761...|
|[6.10338365446613...|
+--------------------+
only showing top 5 rows



In [10]:
from pyspark.ml.feature import PCA

In [11]:
pca = PCA(k=2, inputCol="features")
pca.setOutputCol("PCA")

PCA_f3ea723c475d

In [12]:
model=pca.fit(d)

In [15]:
a=model.transform(d).collect()[0].PCA
a

DenseVector([-96572.9755, -6846.9647])

In [16]:
model.explainedVariance

DenseVector([0.9716, 0.0283])